In [1]:
# Display the current working directory
getwd();
# If necessary, change the path below to the directory where the data files are stored.
# "." means current directory. On Windows use a forward slash / instead of the usual \.
workingDir = "/home/jovyan/work/phd/datasets/gtex/log/3000_random-5000hvg/";
setwd(workingDir);
# Load WGCNA package
library(WGCNA)
# The following setting is important, do not omit.
options(stringsAsFactors = FALSE);

[1] "/home/jovyan/work/phd/R_stuff"

Loading required package: dynamicTreeCut
Loading required package: fastcluster

Attaching package: ‘fastcluster’

The following object is masked from ‘package:stats’:

    hclust



Attaching package: ‘WGCNA’

The following object is masked from ‘package:stats’:

    cor



In [156]:
add_legend <- function(...) {
  opar <- par(fig=c(0, 1, 0, 1), oma=c(0, 0, 0, 0), 
    mar=c(0, 0, 0, 0), new=TRUE)
  on.exit(par(opar))
  plot(0, 0, type='n', bty='n', xaxt='n', yaxt='n')
  legend(...)
}

In [2]:
datExpr <- t(read.csv("mainTable.csv", check.names=FALSE, row.names = "ensg"))
genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples
allTraits <- read.csv("files.dat")

In [ ]:
log1 <- function(x){
    log(x+1)
}
datExpr <- as.data.frame(apply(datExpr,2, log1))
dimnames(datExpr)[[1]] <- samples

In [3]:
datExpr[1:10,1:5]

,ENSG00000237683,ENSG00000268903,ENSG00000225972,ENSG00000237973,ENSG00000229344
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-16NPV-0011-R8a-SM-7EPH8,1.8256777,0.5966363,2.638343,5.127529,3.441379
GTEX-U8T8-0126-SM-3DB94,3.3322045,0.8060293,2.887033,3.707947,2.702703
GTEX-12ZZW-0011-R5a-SM-5DUVN,0.4242214,0.3291599,3.145014,5.883322,3.889573
GTEX-11TT1-1926-SM-5PNYN,1.4809225,0.2293642,2.464704,4.519067,3.000222
GTEX-T6MN-1226-SM-3NMA5,1.8419772,0.5855063,2.634045,4.799914,3.232779
GTEX-Q2AG-0626-SM-2S1PV,3.1363634,0.8073683,2.832625,3.050694,3.086943
GTEX-13CF2-0011-R2a-SM-5L3DC,0.2694163,0.2508365,2.752386,5.040194,3.564732
GTEX-WHSE-1026-SM-57WAX,1.2853684,0.0000000,2.505526,3.752793,2.751110
GTEX-12WSK-2326-SM-5GCOH,1.4151247,0.0000000,2.905260,4.365008,3.096934


In [4]:
head(allTraits)

SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>
GTEX-1117F-0003-SM-58Q7G,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GTEX-1117F-0003-SM-5DWSB,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GTEX-1117F-0003-SM-6WBT7,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GTEX-1117F-0011-R10a-SM-AHZ7F,NA,"B1, A1",,NA,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GTEX-1117F-0011-R10b-SM-CYKQ8,NA,"B1, A1",,7.2,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GTEX-1117F-0226-SM-5GZZ7,0,B1,"2 pieces, ~15% vessel stroma, rep delineated",6.8,Adipose Tissue,Adipose - Subcutaneous,0002190,1214,1125,⋯,14648800,11999300,0.00315785,14669500,50.0354,0.00310538,0.99474,NA,0,50.1944


In [5]:
#labels <- c('primary_site', 'malignacy', 'disease_type', 'stage')
#labels <- c('primary_site', 'secondary_site')
labels <- c('SMTS', 'SMTSD')
datTraits <- allTraits[,labels]
rownames(datTraits) <- allTraits$X
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
NA,NA,NA
NA.1,NA,NA
NA.2,NA,NA
NA.3,NA,NA
NA.4,NA,NA
NA.5,NA,NA


In [6]:
table( dimnames(datExpr)[[1]]==datTraits$file_name)

< table of extent 0 >

In [7]:
y = datTraits$primary_site

In [8]:
meanExpressionBySample=apply( datExpr,1,mean, na.rm=T)
NumberMissingBySample=apply( is.na(data.frame(datExpr)),1, sum)

## 1

In [9]:
gsg = goodSamplesGenes(datExpr, verbose = 3);
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1


[1] TRUE

In [10]:
sampleTree = hclust(dist(datExpr), method = "average");
# Plot the sample tree: Open a graphic output window of size 12 by 9 inches
# The user should change the dimensions if the window is too large or too small.
sizeGrWindow(12,9)
svg("samplehier.svg")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5,
cex.axis = 1.5, cex.main = 2)
dev.off()

png 
  2

In [11]:
# Determine cluster under the line
clust = cutreeStatic(sampleTree, cutHeight = 200, minSize = 10)
table(clust)

clust
   1 
3000 

In [12]:

# clust 1 contains the samples we want to keep.
keepSamples = (clust==1)
datExpr = datExpr[keepSamples, ]
nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples

In [13]:
head(datExpr)

,ENSG00000237683,ENSG00000268903,ENSG00000225972,ENSG00000237973,ENSG00000229344,ENSG00000272512,ENSG00000187608,ENSG00000186891,ENSG00000184163,ENSG00000179403,⋯,ENSG00000215347,ENSG00000100197,ENSG00000205702,ENSG00000172250,ENSG00000183569,ENSG00000073150,ENSG00000188130,ENSG00000205560,ENSG00000008735,ENSG00000251322
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-16NPV-0011-R8a-SM-7EPH8,1.8256777,0.5966363,2.638343,5.127529,3.441379,0.9130856,3.408173,0.3276477,0.9876807,2.529721,⋯,0.2334898,1.5424430,1.7457155,0.52449181,1.2077656,2.4423470,1.9703240,2.976549,3.7873664,3.045474
GTEX-U8T8-0126-SM-3DB94,3.3322045,0.8060293,2.887033,3.707947,2.702703,0.9493386,2.260929,3.4262151,2.5014360,2.598979,⋯,0.4940863,1.5129270,2.0414800,0.15306438,3.1095073,0.5906160,0.9850702,2.238153,0.9880531,3.243764
GTEX-12ZZW-0011-R5a-SM-5DUVN,0.4242214,0.3291599,3.145014,5.883322,3.889573,1.6225516,3.546740,0.2135779,0.1310283,2.735017,⋯,0.4718145,1.2231870,1.5566698,0.38396905,1.1029363,2.6347624,2.4123360,2.915064,3.7954892,3.445852
GTEX-11TT1-1926-SM-5PNYN,1.4809225,0.2293642,2.464704,4.519067,3.000222,0.3462811,2.625393,0.5709795,0.9454611,2.605648,⋯,0.3356147,0.4119772,0.9360934,0.06653652,0.6705436,0.7070501,2.3633979,2.469793,1.5370823,4.373490
GTEX-T6MN-1226-SM-3NMA5,1.8419772,0.5855063,2.634045,4.799914,3.232779,0.5636078,3.548467,0.3337543,0.9353087,2.764431,⋯,0.4473745,4.6624953,5.2289673,0.00000000,0.5359669,1.7488520,0.5514108,3.286161,0.3012151,2.649715
GTEX-Q2AG-0626-SM-2S1PV,3.1363634,0.8073683,2.832625,3.050694,3.086943,2.5809741,4.288677,1.0636757,0.9403978,5.277604,⋯,1.0299765,1.5937150,2.0241931,0.80781426,2.0429072,0.4071969,2.7107133,3.693618,0.9726711,4.111202


In [18]:
rownames(allTraits) <- allTraits$SAMPID
datTraits <- allTraits[keepSamples,labels]
rownames(datTraits) <- rownames(allTraits[keepSamples,labels])
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
GTEX-16NPV-0011-R8a-SM-7EPH8,Brain,Brain - Hypothalamus
GTEX-U8T8-0126-SM-3DB94,Skin,Skin - Sun Exposed (Lower leg)
GTEX-12ZZW-0011-R5a-SM-5DUVN,Brain,Brain - Caudate (basal ganglia)
GTEX-11TT1-1926-SM-5PNYN,Adipose Tissue,Adipose - Visceral (Omentum)
GTEX-T6MN-1226-SM-3NMA5,Liver,Liver
GTEX-Q2AG-0626-SM-2S1PV,Nerve,Nerve - Tibial


In [19]:
collectGarbage();

In [20]:
# Re-cluster samples
sampleTree2 = hclust(dist(datExpr), method = "average")
fontsize=.5
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("samplehiercut.pdf")
plotDendroAndColors(sampleTree2, traitColors,
    groupLabels = labels,
    main = "Sample dendrogram and trait heatmap",
    dendroLabels=FALSE,
    rowText=datTraits,
    rowTextIgnore=labels[[2]]
    )
dev.off()

png 
  2

## 2

In [21]:
# Choose a set of soft-thresholding powers
powers = c(c(1:10), seq(from = 12, to=20, by=2))
# Call the network topology analysis function
sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5)
# Plot the results:
sizeGrWindow(9, 5)
pdf("scaletopology.pdf")
par(mfrow = c(1,2));
cex1 = 0.9;
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")
# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")
dev.off()

pickSoftThreshold: will use block size 3000.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 3000 of 3000


Warning message:
“executing %dopar% sequentially: no parallel backend registered”

   Power SFT.R.sq  slope truncated.R.sq mean.k. median.k. max.k.
1      1   0.5630  1.290          0.455 584.000  600.0000  855.0
2      2   0.0888  0.328          0.379 197.000  201.0000  377.0
3      3   0.0775 -0.279          0.799  87.900   85.1000  207.0
4      4   0.5000 -0.985          0.937  46.400   40.9000  146.0
5      5   0.7090 -1.340          0.978  27.400   21.7000  111.0
6      6   0.7880 -1.520          0.979  17.400   12.5000   87.6
7      7   0.8350 -1.600          0.982  11.700    7.5300   70.3
8      8   0.8720 -1.610          0.990   8.260    4.6500   57.4
9      9   0.8730 -1.670          0.982   6.000    2.9400   48.8
10    10   0.8790 -1.710          0.989   4.490    1.9000   42.3
11    12   0.8830 -1.760          0.993   2.680    0.8490   32.8
12    14   0.8850 -1.780          0.983   1.710    0.4050   26.3
13    16   0.8610 -1.800          0.951   1.150    0.2070   21.6
14    18   0.3500 -2.610          0.349   0.812    0.1090   18.1
15    20   0.3610 -2.550 

png 
  2

In [22]:
net = blockwiseModules(datExpr, power = 6,
TOMType = "unsigned", minModuleSize = 30,
reassignThreshold = 0, mergeCutHeight = 0.15,
numericLabels = TRUE, pamRespectsDendro = FALSE,
saveTOMs = TRUE,
saveTOMFileBase = "wgcna",
verbose = 3)

 Calculating module eigengenes block-wise from all genes
   Flagging genes and samples with too many missing values...
    ..step 1
 ..Working on block 1 .
    TOM calculation: adjacency..
    ..will not use multithreading.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 1 into file wgcna-block.1.RData
 ....clustering..
 ....detecting modules..
 ....calculating module eigengenes..
 ....checking kME in modules..
     ..removing 3 genes from module 1 because their KME is too low.
     ..removing 7 genes from module 2 because their KME is too low.
     ..removing 1 genes from module 3 because their KME is too low.
     ..removing 3 genes from module 4 because their KME is too low.
     ..removing 4 genes from module 5 because their KME is too low.
     ..removing 1 genes from module 6 because their KME is too low.
     ..removing 1 genes from module 7 because their KME

In [23]:
# open a graphics window
sizeGrWindow(12, 9)
pdf("netdendograms.pdf")
# Convert labels to colors for plotting
mergedColors = labels2colors(net$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
"Module colors",
dendroLabels = FALSE, hang = 0.03,
addGuide = TRUE, guideHang = 0.05)
dev.off()

png 
  2

In [24]:
moduleLabels = net$colors
moduleColors = labels2colors(net$colors)
MEs = net$MEs;
geneTree = net$dendrograms[[1]];

## 3

In [25]:
datTraits$SMTS <- match(datTraits$SMTS, unique(datTraits$SMTS))
datTraits$SMTSD <- match(datTraits$SMTSD, unique(datTraits$SMTSD))

In [40]:
datTraits$primary_site <- match(datTraits$primary_site, unique(datTraits$primary_site))
#datTraits$secondary_site <- match(datTraits$secondary_site, unique(datTraits$secondary_site))
datTraits$malignacy <- match(datTraits$malignacy, unique(datTraits$malignacy))
datTraits$disease_type <- match(datTraits$disease_type, unique(datTraits$disease_type))
datTraits$stage <- match(datTraits$stage, unique(datTraits$stage))

ERROR: Error in `$<-.data.frame`(`*tmp*`, primary_site, value = integer(0)): replacement has 0 rows, data has 1000


In [26]:
# Define numbers of genes and samples
nGenes = ncol(datExpr);
nSamples = nrow(datExpr);
# Recalculate MEs with color labels
MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use = "p");
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples);

In [27]:
pdf("moduleTrait.pdf", width=9, height=18)
# Will display correlations and their p-values
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",
signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3));
# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
xLabels = labels,
yLabels = names(MEs),
ySymbols = names(MEs),
colorLabels = FALSE,
colors = blueWhiteRed(50),
textMatrix = textMatrix,
setStdMargins = FALSE,
cex.text = 0.5,
zlim = c(-1,1),
main = paste("Module-trait relationships"))
dev.off()

png 
  2

In [28]:
# Define variable weight containing the weight column of datTrait
weight = as.data.frame(datTraits$SMTS);
names(weight) = "weight"
# names (colors) of the modules
modNames = substring(names(MEs), 3)
geneModuleMembership = as.data.frame(cor(datExpr, MEs, use = "p"));
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples));
names(geneModuleMembership) = paste("MM", modNames, sep="");
names(MMPvalue) = paste("p.MM", modNames, sep="");
geneTraitSignificance = as.data.frame(cor(datExpr, weight, use = "p"));
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples));
names(geneTraitSignificance) = paste("GS.", names(weight), sep="");
names(GSPvalue) = paste("p.GS.", names(weight), sep="");

In [29]:
module = "greenyellow"
column = match(module, modNames);
moduleGenes = moduleColors==module;
sizeGrWindow(7, 7);
pdf("modulemembership.pdf")
par(mfrow = c(1,1));
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
    abs(geneTraitSignificance[moduleGenes, 1]),
    xlab = paste("Module Membership in", module, "module"),
    ylab = "Gene significance for primary_site",
    main = paste("Module membership vs. gene significance\n"),
    cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

png 
  2

In [119]:
# Create the starting data frame
geneInfo0 = data.frame(
moduleColor = moduleColors,
geneTraitSignificance,
GSPvalue)
# Order modules by their significance for weight
modOrder = order(-abs(cor(MEs, weight, use = "p")));
# Add module membership information in the chosen order
for (mod in 1:ncol(geneModuleMembership))
{
oldNames = names(geneInfo0)
geneInfo0 = data.frame(geneInfo0, geneModuleMembership[, modOrder[mod]],
MMPvalue[, modOrder[mod]]);
names(geneInfo0) = c(oldNames, paste("MM.", modNames[modOrder[mod]], sep=""),
paste("p.MM.", modNames[modOrder[mod]], sep=""))
}
# Order the genes in the geneInfo variable first by module color, then by geneTraitSignificance
geneOrder = order(geneInfo0$moduleColor, -abs(geneInfo0$GS.weight));
geneInfo = geneInfo0[geneOrder, ]
write.csv(geneInfo, file = "geneInfo.csv")

In [120]:
# Read in the probe annotation
annot = read.csv(file = "geneInfo.csv");
# Match probes in the data set to the probe IDs in the annotation file
probes = names(datExpr)
allLLIDs = probes
# $ Choose interesting modules
intModules = c("greenyellow", "darkorange", "darkgreen")
for (module in intModules)
{
# Select module probes
modGenes = (moduleColors==module)
# Get their entrez ID codes
modLLIDs = probes[modGenes];
# Write them into a file
fileName = paste("LocusLinkIDs-", module, ".txt", sep="");
write.table(as.data.frame(modLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)
}
# As background in the enrichment analysis, we will use all probes in the analysis.
fileName = paste("LocusLinkIDs-all.txt", sep="");
write.table(as.data.frame(allLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)

In [121]:
require("biomaRt")
mart <- useMart("ENSEMBL_MART_ENSEMBL")
mart <- useDataset("hsapiens_gene_ensembl", mart)

ens <- allLLIDs
ensLookup <- gsub("\\.[0-9]*$", "", ens)

annotLookup <- getBM(
  mart=mart,
  attributes=c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"),
  filter="ensembl_gene_id",
  values=ensLookup,
  uniqueRows=TRUE)

annotLookup <- data.frame(
  ens[match(annotLookup$ensembl_gene_id, ensLookup)],
  annotLookup)

colnames(annotLookup) <- c(
  "original_id",
  c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"))

annotLookup

Loading required package: biomaRt


ERROR: Error in .processResults(postRes, mart = mart, sep = sep, fullXmlQuery = fullXmlQuery, : Query ERROR: caught BioMart::Exception::Database: Could not connect to mysql database ensembl_mart_99: DBI connect('database=ensembl_mart_99;host=127.0.0.1;port=5316','ensro',...) failed: Can't connect to MySQL server on '127.0.0.1' (111) at /net/isilonP/public/ro/ensweb/live/mart/www_99/biomart-perl/lib/BioMart/Configuration/DBLocation.pm line 98.



## 5 

In [157]:
# Re-cluster samples
sampleTree3 = hclust(dist(MEs0), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
svg("nethier.svg", width=10)
# Set the plot margin: bottom, left, top & right
par(mar = c(5, 4, 1.4, 0.2) + 0.1,
    xpd = NA) # allow content to go into outer margin 
plotDendroAndColors(sampleTree3, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE,
#rowText=allTraits[keepSamples,labels],
#rowTextIgnore=labels[[2]],
addGuide=TRUE)
dev.off()

png 
  2

In [159]:
svg("nethier_legend.svg", width=10)
add_legend("topleft", legend = unique(allTraits[keepSamples,'SMTS']), pch = 1, pt.cex = 1, cex = 1.5, bty = 'n', ncol=2,
       inset = c(0.1, 0.), # place outside
       title = "SMTS", 
       col = traitColors)
dev.off()

Warning message in par(fig = c(0, 1, 0, 1), oma = c(0, 0, 0, 0), mar = c(0, 0, 0, :
“calling par(new=TRUE) with no plot”

png 
  2

In [43]:
collectGarbage()

In [31]:
clust = cutreeStatic(sampleTree3, cutHeight = 0.4, minSize = 10)
table(clust)

clust
   1 
3000 

In [32]:
# Re-cluster samples after cut
sampleTree4 = hclust(dist(MEs0[(clust==1),]), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits[(clust==1),]);
# Plot the sample dendrogram and the colors underneath.
svg("nethiercut.svg")
plotDendroAndColors(sampleTree4, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE)
dev.off()

png 
  2

In [33]:
# Calculate topological overlap anew: this could be done more efficiently by saving the TOM
# calculated during module detection, but let us do it again here.
dissTOM = 1-TOMsimilarityFromExpr(datExpr, power = 6);
# Transform dissTOM with a power to make moderately strong connections more visible in the heatmap
plotTOM = dissTOM^7;
# Set diagonal to NA for a nicer plot
diag(plotTOM) = NA;
# Call the plot function
sizeGrWindow(9,9)
svg("networkmap.svg")
TOMplot(plotTOM, geneTree, moduleColors, main = "Network heatmap plot, all genes")
dev.off()

TOM calculation: adjacency..
..will not use multithreading.
 Fraction of slow calculations: 0.000000
..connectivity..
..matrix multiplication (system BLAS)..
..normalization..
..done.


png 
  2

In [34]:
#selct genes to be faster
nSelect = nGenes*0.7
# For reproducibility, we set the random seed
set.seed(10);
select = sample(nGenes, size = nSelect);
selectTOM = dissTOM[select, select];
# There’s no simple way of restricting a clustering tree to a subset of genes, so we must re-cluster.
selectTree = hclust(as.dist(selectTOM), method = "average")
selectColors = moduleColors[select];
# Open a graphical window
sizeGrWindow(9,9)
svg("networkmap.svg")
# Taking the dissimilarity to a power, say 10, makes the plot more informative by effectively changing
# the color palette; setting the diagonal to NA also improves the clarity of the plot
plotDiss = selectTOM^7;
diag(plotDiss) = NA;
TOMplot(plotDiss, selectTree, selectColors, main = "Network heatmap plot, selected genes")
dev.off()

png 
  2

In [37]:
# Recalculate module eigengenes
MEs = moduleEigengenes(datExpr, moduleColors)$eigengenes
# Isolate weight from the clinical traits
weight = as.data.frame(datTraits$SMTS);
names(weight) = "SMTS"
# Add the weight to existing module eigengenes
MET = orderMEs(cbind(MEs, weight))
# Plot the relationships among the eigengenes and the trait
sizeGrWindow(5,7.5);
svg("eigenHeat.svg")
par(cex = 0.9)
plotEigengeneNetworks(MET, "", marDendro = c(0,4,1,2), marHeatmap = c(3,4,1,2), cex.lab = 0.8, xLabelsAngle = 90)
dev.off()

png 
  2

In [38]:
# Plot the dendrogram
sizeGrWindow(6,6);
svg("eigendendogram.svg")
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene dendrogram", marDendro = c(0,4,2,0),
plotHeatmaps = FALSE)
# Plot the heatmap matrix (note: this plot will overwrite the dendrogram plot)
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene adjacency heatmap", marHeatmap = c(3,4,2,2),
plotDendrograms = FALSE, xLabelsAngle = 90)
dev.off()

png 
  2

In [134]:
write.csv(cutree(sampleTree3, k=3), "wgcna_level_3_labels.csv")

In [135]:
write.csv(geneModuleMembership, "wgcna_level_0_word-dist.csv")

In [136]:
write.csv(MEs, "wgcna_level_0_topic-dist.csv")